In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from intent_classification_helper import *

In [ ]:
atis_train = pd.read_csv('/content/drive/MyDrive/nlp_datasets/ATIS dataset/atis_intents_train.csv', error_bad_lines=False, 
                   engine='python', encoding='utf-8', names=['intent', 'message'])
atis_test = pd.read_csv('/content/drive/MyDrive/nlp_datasets/ATIS dataset/atis_intents_test.csv', error_bad_lines=False, 
                   engine='python', encoding='utf-8', names=['intent', 'message'])
assert(set(atis_train['intent']) == set(atis_test['intent']))
atis_train = balance_class(atis_train, 'message', 'intent')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
atis_train.shape

(29328, 2)

In [ ]:
atis_test.shape

(800, 2)

In [ ]:
# encode label
le = LabelEncoder()
atis_train['intent'] = le.fit_transform(atis_train['intent'])
atis_test['intent'] = le.fit_transform(atis_test['intent'])

onehot_encoder = OneHotEncoder()
y_train = onehot_encoder.fit_transform(atis_train['intent'].values.reshape(-1, 1)).todense()
y_test = onehot_encoder.fit_transform(atis_test['intent'].values.reshape(-1, 1)).todense()

In [ ]:
# preprocess text
print('Preprocessing text on training set...')
preprocess_text(atis_train, 'message')

print('Preprocessing text on test set...')
preprocess_text(atis_test, 'message')

x_train = atis_train['message'].tolist()
x_test = atis_test['message'].tolist()

Preprocessing text on training set...


Start text preprocessing: 
--------------------------
Converting to lowercase...
--------------------------
Removing html tags...
--------------------------
Removing nonword characters...
--------------------------
Removing stopwords...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Text preprocessing completed.


Preprocessing text on test set...


Start text preprocessing: 
--------------------------
Converting to lowercase...
--------------------------
Removing html tags...
--------------------------
Removing nonword characters...
--------------------------
Removing stopwords...
Text preprocessing completed.




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# try different number of features
#tv = TfidfVectorizer(max_df=1.0, min_df=0, max_features=40)
#tv = TfidfVectorizer(max_df=1.0, min_df=0, max_features=100)
#tv = TfidfVectorizer(max_df=1.0, min_df=0, max_features=200)
#tv = TfidfVectorizer(max_df=1.0, min_df=0, max_features=1000)
tv = TfidfVectorizer(max_df=1.0, min_df=0)

tv.fit(x_train)
x_train = tv.transform(x_train).toarray()
x_test = tv.transform(x_test).toarray()

vocab = tv.get_feature_names_out()

print(pd.DataFrame(x_train, columns=vocab))
print('TF-IDF vocabulary size: ', len(vocab))

       0900   10  100      1000  1020  1024  1026  1030  1039  1045  ...  \
0       0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1       0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   
2       0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3       0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4       0.0  0.0  0.0  0.504342   0.0   0.0   0.0   0.0   0.0   0.0  ...   
...     ...  ...  ...       ...   ...   ...   ...   ...   ...   ...  ...   
29323   0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   
29324   0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   
29325   0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   
29326   0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   
29327   0.0  0.0  0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  ...   

       without  working  world  worth  would  year  yes   yn  york  yyz  
0          0.

In [ ]:
x_train.shape

(29328, 748)

In [ ]:
from keras.layers import MaxPooling1D

In [ ]:
def cnn_atis(x_train_cnn, y_train_cnn, batch_size, epochs, validation_data, feature_numbers):
  model = Sequential()
  # model.add(Embedding(input_dim=feature_numbers, output_dim=64, input_length=feature_numbers))
  # model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
  model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(x_train_cnn.shape[1], 1)))

  model.add(BatchNormalization())
  model.add(Dropout(0.5)) 
  model.add(MaxPooling1D(2))
  model.add(Flatten())

  model.add(Dense(units=100, activation='relu'))
  model.add(Dense(units=8, activation='softmax'))
  model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])
  model.summary()
  # path = '/content/drive/MyDrive/nlp_datasets/CLINC150/models/'
  # path = os.path.join(path, 'cnn_{}_features'.format(feature_numbers))
  # if not os.path.isdir(path):
  #   os.mkdir(path)

  #checkpoint = ModelCheckpoint(filepath=path, monitor='val_accuracy')

  #model.fit(x_train_cnn, y_train_cnn, batch_size=batch_size, epochs=epochs, validation_data=validation_data, callbacks=[checkpoint])
  #model.fit(x_train_cnn, y_train_cnn, batch_size=batch_size, epochs=epochs, validation_data=validation_data)

  return model

In [ ]:
# running cnn without word embedding layer
model = cnn_atis(x_train_cnn=x_train, y_train_cnn=y_train, batch_size=8, epochs=20, validation_data=(x_test, y_test), feature_numbers=len(vocab))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 746, 32)           128       
                                                                 
 batch_normalization_1 (Batc  (None, 746, 32)          128       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 746, 32)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 373, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 11936)             0         
                                                                 
 dense (Dense)               (None, 100)              

In [ ]:
model.fit(x_train, y_train, batch_size=8, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
3666/3666 [==============================] - 71s 19ms/step - loss: 0.0697 - accuracy: 0.9801 - val_loss: 0.1575 - val_accuracy: 0.9550
Epoch 2/20
3666/3666 [==============================] - 69s 19ms/step - loss: 0.0185 - accuracy: 0.9946 - val_loss: 0.2106 - val_accuracy: 0.9375
Epoch 3/20
3666/3666 [==============================] - 68s 19ms/step - loss: 0.0125 - accuracy: 0.9969 - val_loss: 0.3337 - val_accuracy: 0.9488
Epoch 4/20
3666/3666 [==============================] - 68s 18ms/step - loss: 0.0120 - accuracy: 0.9970 - val_loss: 0.2272 - val_accuracy: 0.9538
Epoch 5/20
3666/3666 [==============================] - 68s 19ms/step - loss: 0.0062 - accuracy: 0.9985 - val_loss: 0.1921 - val_accuracy: 0.9650
Epoch 6/20
3666/3666 [==============================] - 68s 18ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 0.4999 - val_accuracy: 0.9425
Epoch 7/20
3666/3666 [==============================] - 69s 19ms/step - loss: 0.0076 - accuracy: 0.9982 - val_loss: 0.1927 -